In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q flask pyngrok huggingface-hub
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124 --force-reinstall --no-deps
!pip install typing-extensions numpy diskcache jinja2

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00


In [3]:
!nvidia-smi


Thu Dec 11 09:16:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from pyngrok import ngrok
import time
import llama_cpp
from flask import Flask, request, jsonify
import subprocess

print(llama_cpp.llama_supports_gpu_offload())

# Load model
llm = llama_cpp.Llama.from_pretrained(
    repo_id="mradermacher/Qwen2.5-7B-Instruct-Uncensored-GGUF",
    filename="Qwen2.5-7B-Instruct-Uncensored.Q4_K_S.gguf",
    cache_dir="/content/drive/MyDrive/models",
    n_ctx=4096,
    n_threads=2,
    n_gpu_layers=-1,
    verbose=False
)

print(f"Model metadata: {llm.metadata}")
print("Model loaded successfully")


# Set up Flask API
ngrok.set_auth_token("35Q63lZ04yGZ6gkAQlgYe6T4gDD_bukh6YjAjq9ZZsoJEkGE")

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    print("Got request")
    start_time = time.time()

    data = request.json
    prompt = data.get('prompt', '')
    craziness_level = int(data.get('craziness', 0))

    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400

    # Build the ChatML formatted prompt
    full_prompt = data.get('prompt', '')

    print(f"Craziness level: {craziness_level}")
    # print(f"Full prompt: {full_prompt}")
    response = llm(
        full_prompt,
        max_tokens=150,
        temperature=0.6,
        top_p=0.9,
        repeat_penalty=1.1,
        stop=["<|im_end|>", "<|im_start|>", "Patient:", "User:"]
    )

    # Log GPU usage
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.used,utilization.gpu', '--format=csv,noheader,nounits'],
        capture_output=True, text=True
    )
    print(f"GPU: {result.stdout.strip()}")

    generated_text = response['choices'][0]['text'].strip()

    print(f"Response ({time.time() - start_time:.2f}s): {generated_text[:100]}...")

    return jsonify({'generated_text': generated_text})


@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'ready'})


public_url = ngrok.connect(5000)
print(f"\nAPI running at: {public_url}")

app.run(port=5000)

True


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Model metadata: {'mradermacher.quantized_on': 'db3', 'mradermacher.quantized_at': '2024-10-11T20:11:44+02:00', 'mradermacher.quantized_by': 'mradermacher', 'mradermacher.quantize_version': '2', 'general.base_model.0.repo_url': 'https://huggingface.co/Qwen/Qwen2.5-7B-Instruct', 'general.type': 'model', 'qwen2.block_count': '28', 'general.base_model.0.name': 'Qwen2.5 7B Instruct', 'tokenizer.ggml.pre': 'qwen2', 'mradermacher.convert_type': 'hf', 'general.source.url': 'https://huggingface.co/Orion-zhen/Qwen2.5-7B-Instruct-Uncensored', 'general.base_model.count': '1', 'general.base_model.0.organization': 'Qwen', 'general.size_label': '7B', 'tokenizer.ggml.add_bos_token': 'false', 'general.basename': 'Qwen2.5', 'qwen2.embedding_length': '3584', 'tokenizer.ggml.padding_token_id': '151643', 'qwen2.context_length': '32768', 'general.architecture': 'qwen2', 'general.url': 'https://huggingface.co/mradermacher/Qwen2.5-7B-Instruct-Uncensored-GGUF', 'qwen2.feed_forward_length': '18944', 'tokenizer.

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Got request
Craziness level: 0


...

GPU: 8614, 90
Response (1.88s): [GESTURE: hey] How are you feeling today? Let's dive into what's been on your mind....
Got request
Craziness level: 2


INFO:werkzeug:127.0.0.1 - - [11/Dec/2025 10:53:49] "POST /generate HTTP/1.1" 200 -


GPU: 8614, 89
Response (3.05s): [GESTURE: fast_nod] Of course I can hear you! Just a thought, maybe try to speak louder next time. [...
Got request
Craziness level: 3


INFO:werkzeug:127.0.0.1 - - [11/Dec/2025 10:54:09] "POST /generate HTTP/1.1" 200 -


GPU: 8614, 87
Response (3.55s): [GESTURE: nod] Sure thing buddy! Just remember, speaking louder can really make a difference. And wh...
